# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-16 13:17:24] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-16 13:17:24] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-16 13:17:24] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-16 13:17:27] INFO server_args.py:1657: Attention backend not specified. Use fa3 backend by default.


[2026-01-16 13:17:27] INFO server_args.py:2556: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=76.93 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=80 avail_mem=76.81 GB):  20%|██        | 4/20 [00:05<00:17,  1.09s/it]

Capturing batches (bs=32 avail_mem=76.78 GB):  50%|█████     | 10/20 [00:06<00:03,  3.20it/s]

Capturing batches (bs=4 avail_mem=76.75 GB):  80%|████████  | 16/20 [00:06<00:00,  6.66it/s]

Capturing batches (bs=1 avail_mem=76.74 GB): 100%|██████████| 20/20 [00:06<00:00,  3.17it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Michael, I am the owner of a small business and I recently decided to change my brand name. I am looking to create a new business name. Which of the following is the best way to do this? I have not decided what my business will be called yet. I have a name that I have already created and I would like to change it to a new name. To properly start the process of changing your business name, it's important to first understand your business's history and values. This will help you create a name that reflects your brand and provides clarity to your potential customers and employees. 

In addition, it's important to
Prompt: The president of the United States is
Generated text:  trying to decide how many military bases to have. He believes that the best military bases are located near the coasts. He has collected data from various countries on the number of military bases per capita. 

A) The president looks at data from Russia, where the population 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I am a [Age] year old [Occupation]. I have always been fascinated by the world around me and have always been curious about the mysteries of the universe. I have a passion for learning and always strive to expand my knowledge and understanding of the world. I am always looking for new experiences and adventures, and I am always eager to learn new things. I am a [Favorite Hobby] person and I enjoy spending time with my family and friends. I am also a [Favorite Book] person and I love to read and learn from the books that inspire me. I am a [Favorite Movie] person and

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. 

A. True
B. False
A. True

Paris is the capital city of France, located on the Seine River in the Loire Valley. It is the largest city in France by population and is known for its rich history, art, and culture. The city is home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is also a major center for business, finance, and tourism, making it a major economic and cultural hub in France. 

Therefore, the statement "The capital of France is Paris" is true. However, it

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction. Here are some of the most likely trends:

1. Increased use of AI in healthcare: AI is already being used in healthcare to improve patient outcomes and reduce costs. As the technology continues to advance, we can expect to see even more widespread use of AI in healthcare, from personalized medicine to predictive analytics.

2. AI in finance: AI is already being used in finance to improve fraud detection, risk management, and portfolio optimization. As the technology continues to evolve, we can expect to see even more widespread use of AI in finance, from automated



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [insert name] and I am a [insert occupation or profession] with a [insert relevant skill or experience] experience. I have always been a fan of [insert favorite genre or subject] and I enjoy [insert interests or hobbies]. What is your favorite way to spend your free time? As an AI language model, I am here to assist you with any questions or tasks you may have. How can I help you today? 
Let me know if you have any questions or need further clarification. I am here to assist you in any way I can. 
[Insert name] (insert number of words) 
Hello, my

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the largest city and the most populous city in the European Union, located on the Seine River and having a history dating back to the Roman Empire. It is a historical and cultural center, with many iconic landmar

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 applications

.



3

.

 AI

In [6]:
llm.shutdown()